Let's discover relations between natures and each agency.

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import math

# Load data

In [2]:
calls = pd.read_csv('../data/callsforservicefiltered.csv')
ems_calls = pd.DataFrame()
gcf_calls = pd.DataFrame()
gcsd_calls = pd.DataFrame()
aco_calls = pd.DataFrame()

/home/oana/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Let's clean it up.

In [ ]:
calls = calls[calls.rptonly != True]
calls = calls[calls.cancelled != True]
calls = calls[calls.nature != 'TRANSPORT TO ANIMAL SHELTER']
calls['calltime'] = pd.to_datetime(calls.calltime)
calls['year'] = calls.calltime.apply(lambda x: x.date().year)
calls['month'] = calls.calltime.apply(lambda x: x.date().month)
calls['dayOfMonth'] = calls.calltime.apply(lambda x: x.date().day)
calls['hour'] = calls.calltime.apply(lambda x: x.time().hour)
calls = calls.rename(columns={'calldow' : 'dayOfWeek'})
calls = calls.drop(columns = ['timeclose', 'calltime'], axis = 1)
calls = calls.drop(columns = ['statbeat','ra'])
calls['actualnature'] = [x['nature2'] if x['nature'] is np.nan and x['nature']!=x['nature2']
                        else x['nature'] for i,x in calls.iterrows()]
calls = calls.drop(columns = ['nature','nature2'])
calls = calls.rename(columns={'actualnature' : 'nature'})

Let's split the data into 4 data sets based on the 4 agencies.

In [ ]:
ems_calls = calls[calls.agency == 'EMS']
aco_calls = calls[calls.agency == 'ACO']
gcsd_calls = calls[calls.agency == 'GCSD']
gcf_calls = calls[calls.agency == 'GCF']

## 1. First 5 leading natures for calls per agency
Let's see what are the top reasons per agency most people call about.

Let's group all tuples into groups based on the same call nature

In [ ]:
# warning: inPlace parameter has a bug! Do not use it in sort_values(), as the object returned is Null
aco_groupby_nature = aco_calls.groupby(['nature']).size().reset_index(name='count')
aco_groupby_nature = aco_groupby_nature.sort_values(by='count', ascending=False)
ems_groupby_nature = ems_calls.groupby(['nature']).size().reset_index(name='count')
ems_groupby_nature = ems_groupby_nature.sort_values(by='count', ascending=False)
gcf_groupby_nature = gcf_calls.groupby(['nature']).size().reset_index(name='count')
gcf_groupby_nature = gcf_groupby_nature.sort_values(by='count', ascending=False)
gcsd_groupby_nature = gcsd_calls.groupby(['nature']).size().reset_index(name='count')
gcsd_groupby_nature = gcsd_groupby_nature.sort_values(by='count', ascending=False)

Print the top 5 calls' nature in each call category and see which is the biggest.

In [ ]:
def create_plot(ylabel, title, xlabels, xsizes):
    
    y_pos = np.arange(len(xlabels))    
    plt.bar(y_pos, xsizes, align='center', alpha=1)
    plt.xticks(y_pos, xlabels, rotation='vertical')
    plt.ylabel(ylabel)
    plt.title(title)
    plt.show()

In [ ]:
font = {'family':'normal',
#         'weight':'bold',
       'size':5}
# matplotlib.rc('font', **font)
print('ACO top 5 reasons for calls')
print(aco_groupby_nature.head())
aco_top_natures = aco_groupby_nature.head(10)['nature']
no_calls_aco = aco_groupby_nature.head(10)['count']
create_plot('Number of calls', 'ACO top reasons for calls', 
            aco_top_natures, no_calls_aco)

print('\nEMS top 5 reasons for calls')
print(ems_groupby_nature.head())
ems_top_natures = ems_groupby_nature.head(10)['nature']
no_calls_ems = ems_groupby_nature.head(10)['count']
create_plot('Number of calls', 'EMS top reasons for calls', 
            ems_top_natures, no_calls_ems)

print('\nGCF top 5 reasons for calls')
print(gcf_groupby_nature.head())
gcf_top_natures = gcf_groupby_nature.head(10)['nature']
no_calls_gcf = gcf_groupby_nature.head(10)['count']
create_plot('Number of calls', 'GCF top reasons for calls', 
            gcf_top_natures, no_calls_gcf)

print('\nGCSD top 5 reasons for calls')
print(gcsd_groupby_nature.head())
gcsd_top_natures = gcsd_groupby_nature.head(10)['nature']
no_calls_gcsd = gcsd_groupby_nature.head(10)['count']
create_plot('Number of calls', 'GCSD top reasons for calls', 
            gcsd_top_natures, no_calls_gcsd)

## 2. Natures with the biggest variance in calls
Let's see which natures have the biggest variance in calls - aka which one are trending.
We will do the computation if the nature is present for at least 3 years, to avoid biased results.
Also we will remove calls that were made in 2018 or 2012, as the data for those years is incomplete and we will obtain biased results.

In [ ]:
def find_call_variance(data):

    agency = data['agency'][0] 
    data_filtered = data[data.year != 2018]
    data_filtered = data_filtered[data_filtered.year != 2012]    
    data_calls_variance = pd.DataFrame(columns=['agency', 'nature', 'variance'])          
    data_with_counts = data_filtered.groupby(['nature', 'year']).size().reset_index(name='count')
    data_with_counts = data_with_counts.sort_values(by='year', ascending=True)
    grouped_data = data_with_counts.groupby(['nature'])
    natures = []
    variance_calls = []    
    for group in grouped_data: 
        calls_group = group[1]
        mean = calls_group['count'].mean()         
        if (mean>50) and (calls_group.tail(1).year == 2017).bool() and calls_group.shape[0] > 2:                        
            calls_2017 = calls_group['count'].tail(1)
            variance_call = (int)(calls_2017-mean)/mean
            natures.append(group[0])
            variance_calls.append(variance_call)        
            
    data_calls_variance['nature'] = natures
    data_calls_variance['variance'] = variance_calls
    return data_calls_variance

First group the data by nature and year and obtain the no of calls for each group.
Then, for each call nature, see how the last year's calls varies according to the mean of all the calls from all the years for that nature.
Then sort the data and print the first 3 biggest differences.
Notes:
3. Compute values for natures with at least x calls

In [ ]:
aco_calls.reset_index(drop=True, inplace=True)
aco_variance = find_call_variance(aco_calls)
aco_variance.sort_values(by='variance', ascending=False, inplace=True)
aco_variance['agency'] = 'ACO'
print('ACO variance')
print(aco_variance.head(3))
aco_trending_natures = aco_variance.head(10)['nature']
aco_variance = aco_variance.head(10)['variance']
create_plot('Variance', 'ACO top trending natures', 
            aco_trending_natures, aco_variance)

ems_calls.reset_index(drop=True, inplace=True)
ems_variance = find_call_variance(ems_calls)
ems_variance.sort_values(by='variance', ascending=False, inplace=True)
ems_variance['agency'] = 'EMS'
print('EMS variance')
print(ems_variance.head(3))
ems_trending_natures = ems_variance.head(10)['nature']
ems_variance = ems_variance.head(10)['variance']
create_plot('Variance', 'EMS top trending natures', 
            ems_trending_natures, ems_variance)

gcsd_calls.reset_index(drop=True, inplace=True)
gcsd_variance = find_call_variance(gcsd_calls)
gcsd_variance.sort_values(by='variance', ascending=False, inplace=True)
gcsd_variance['agency'] = 'GCSD'
print('GCSD variance')
print(gcsd_variance.head(3))
gcsd_trending_natures = gcsd_variance.head(10)['nature']
gcsd_variance = gcsd_variance.head(10)['variance']
create_plot('Variance', 'GCSD top trending natures', 
            gcsd_trending_natures, gcsd_variance)

gcf_calls.reset_index(drop=True, inplace=True)
gcf_variance = find_call_variance(gcf_calls)
gcf_variance.sort_values(by='variance', ascending=False, inplace=True)
gcf_variance['agency'] = 'GCF'
print('GCF variance')
print(gcf_variance.head(3))
gcf_trending_natures = gcf_variance.head(10)['nature']
gcf_variance = gcf_variance.head(10)['variance']
create_plot('Variance', 'GCF top trending natures', 
            gcf_trending_natures, gcf_variance)
#if you want to keep hierarchical indexing, to retrieve data by chunks:
#result = pd.concat(frames, keys=['x', 'y', 'z'])